In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [ ]:
import pandas as pd
from urllib import request
from DateProvider import DateProvider
from datetime import datetime
from HtmlTransformerUtil import HtmlTransformerUtil
from ColumnsAdder import ColumnsAdder
from KreisOptionsSetter import KreisOptionsSetter
from TestHelper import TestHelper
from IOUtils import IOUtils
from Datawrapper import Datawrapper
from MedianOfFreeBedsByKreisTableFactory import MedianOfFreeBedsByKreisTableFactory
from TimeseriesReader import readTimeseries

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', 'raise')


In [ ]:
print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

In [ ]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())        
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

In [ ]:
def saveLastUpdatedIntensivstationen(lastUpdated):
    def setLastUpdated(soup):
        soup.find(id = "Datenstand").string.replace_with(lastUpdated.strftime(DateProvider.INTENSIVSTATIONEN_DATE_FORMAT))
        return soup

    HtmlTransformerUtil().applySoupTransformerToFile(
        file = "../../docs/intensivstationen.html",
        soupTransformer = setLastUpdated)

In [ ]:
saveLastUpdatedIntensivstationen(dateProvider.getLastUpdatedDataSource())

In [ ]:
timeSeries = readTimeseries(download = needsUpdate)
timeSeries

In [ ]:
def readKreise(download = False):
    kreiseFile = '04-kreise.xlsx'
    if download:
        _downloadKreise(kreiseFile)
    
    kreise = pd.read_excel(
        kreiseFile,
        sheet_name = 'Kreisfreie Städte u. Landkreise',
        header = 5,
        index_col = 0)
    kreise = kreise.rename(columns = {'2': 'Bundesland', 3: 'Kreis', 6: 'Einwohnerzahl'})[['Bundesland', 'Kreis', 'Einwohnerzahl']]
    kreise.index.set_names("Key", inplace = True)
    return kreise

# download https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.xlsx?__blob=publicationFile or https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.html
def _downloadKreise(kreiseFile):
    request.urlretrieve(
        'https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.xlsx?__blob=publicationFile',
        kreiseFile)


In [ ]:
kreise = readKreise(download = False)
kreise

In [ ]:
timeSeries = ColumnsAdder(kreise).addKreisAndBundeslandAndEinwohnerzahlColumns(timeSeries)
timeSeries

In [ ]:
kreisValues = sorted(timeSeries['Kreis'].drop_duplicates().values)

In [ ]:
def getKreisOptions(kreisValues):
    return [getKreisOption(kreis) for kreis in kreisValues]

def getKreisOption(kreis):
    return f'<option value="{kreis}">{kreis}</option>'


In [ ]:
kreisOptions = ['<option selected="" value="de">Alle Landkreise</option>']  + getKreisOptions(kreisValues)

In [ ]:
from bs4 import BeautifulSoup


def saveKreisOptions(kreisOptions):
    HtmlTransformerUtil().applySoupTransformerToFile(
        file = "../../docs/intensivstationen.html",
        soupTransformer =
            lambda soup:
                BeautifulSoup(
                    KreisOptionsSetter().setKreisOptions(html = str(soup), options = kreisOptions),
                    'lxml'))


In [ ]:
saveKreisOptions(kreisOptions)

In [ ]:
def getIntensiveCareBeds(timeSeries, kreis = None):
    if kreis is not None:
        return timeSeries[timeSeries['Kreis'] == kreis][['date', 'betten_belegt', 'betten_frei', 'Einwohnerzahl']]
    else:
        return timeSeries.groupby('date').agg(**{
                        'betten_belegt': pd.NamedAgg(column = 'betten_belegt', aggfunc = 'sum'),
                        'betten_frei':   pd.NamedAgg(column = 'betten_frei',   aggfunc = 'sum'),
                        'Einwohnerzahl': pd.NamedAgg(column = 'Einwohnerzahl', aggfunc = 'sum')
                    }).reset_index()

In [ ]:
def getAndPersistIntensiveCareBeds(timeSeries, kreis = None):
    intensiveCareBeds = getIntensiveCareBeds(timeSeries, kreis)
    display(kreis)
    _saveAsJson(intensiveCareBeds, _getFilename(kreis))
    return intensiveCareBeds


def _saveAsJson(intensiveCareBeds, file):
    IOUtils.saveDictAsJson(
        {
            'population': int(intensiveCareBeds.iloc[0]['Einwohnerzahl']),
            'data': _intensiveCareBeds2Dict(intensiveCareBeds),
        },
        file)


def _intensiveCareBeds2Dict(intensiveCareBeds):
    df = intensiveCareBeds[['date', 'betten_belegt', 'betten_frei']].copy()
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    return df.to_dict(orient = "records")


def _getFilename(kreis):
    return f'../../docs/data/intensivstationen/intensivstationen-{_getSuffix(kreis)}.json'


def _getSuffix(kreis):
    return kreis if kreis is not None else 'de'


In [ ]:
getAndPersistIntensiveCareBeds(timeSeries)

In [ ]:
for kreis in kreisValues:
    getAndPersistIntensiveCareBeds(timeSeries, kreis)

In [ ]:
medianOfFreeBedsByKreisTableFactory = MedianOfFreeBedsByKreisTableFactory(timeSeries)
medianOfFreeBedsByKreisTable = medianOfFreeBedsByKreisTableFactory.createMedianOfFreeBedsByKreisTable('Kreis')
medianOfFreeBedsByKreisTable.reset_index().to_json('../../docs/data/intensivstationen/medianOfFreeBedsByKreisTable.json', orient = "records")

In [ ]:
def createMedianOfFreeBedsByKreisTableForChoroplethMap(medianOfFreeBedsByKreisTableFactory):
    medianOfFreeBedsByKreisTable = medianOfFreeBedsByKreisTableFactory.createMedianOfFreeBedsByKreisTable('gemeindeschluessel').reset_index()
    return ColumnsAdder(kreise).addKreisAndEinwohnerzahlColumns(medianOfFreeBedsByKreisTable)
    
medianOfFreeBedsByKreisTable = createMedianOfFreeBedsByKreisTableForChoroplethMap(medianOfFreeBedsByKreisTableFactory)

In [ ]:
def getChartTitle(dateStart, dateEnd):
    def formatDate(date):
        return date.strftime("%d.%m.%Y")

    return f"Median freier Intensivbetten im Zeitraum {formatDate(dateStart)} bis {formatDate(dateEnd)}"

In [ ]:
from decouple import AutoConfig

config = AutoConfig(search_path='../..')
dataWrapper = Datawrapper(config('DATAWRAPPER_API_TOKEN'))
dataWrapper.setChartTitle(
    getChartTitle(
        dateStart=timeSeries['date'].min(),
        dateEnd=timeSeries['date'].max()))
dataWrapper.uploadChartData(medianOfFreeBedsByKreisTable)
dataWrapper.publishChart()


In [ ]:
def publishGitHubPages():
    %cd /home/frankknoll/Dokumente/Corona/projects/HowBadIsMyBatch-pages
    ! git add -A
    ! git commit -m "updating data for Intensivstationen"
    ! git push

### see https://knollfrank.github.io/HowBadIsMyBatch/intensivstationen.html

In [ ]:
publishGitHubPages()